In [37]:
import os
import ast
import json
import time
import pandas as pd
from dotenv import load_dotenv
import prompt_templates_summarization as pts
from langchain.schema import BaseOutputParser
from langchain.prompts.chat import ChatPromptTemplate
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory
)
from json.decoder import JSONDecodeError
from google.generativeai.types import BlockedPromptException
from google.generativeai.types.generation_types import StopCandidateException

In [36]:
country = "Hungary"
path2SP = "/Users/ctoruno/OneDrive - World Justice Project/EU Subnational"

## Loading API key

In [38]:
load_dotenv()
GoogleAI_key = os.getenv("googleAI_API_key")
os.environ['GOOGLE_API_KEY'] = GoogleAI_key

## Reading the data

In [39]:
def gatherFiles(country, p):
    """
    This function takes a country as input and returns a list with all the news articles associated to that specific pillar.
    """
    data_path = f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-summarization/{country}/pillar_{p}"
    sets = [pd.read_parquet(f"{data_path}/{x}") for x in os.listdir(data_path)]
    pillar_data = pd.concat(sets)
    pillar_data["associated_pillar"] = f"Pillar {p}"
    print(f"Pillar {p}: {len(pillar_data)} articles")

    return pillar_data

In [40]:
country_data = [gatherFiles(country, p) for p in range(1,9)]

Pillar 1: 2196 articles
Pillar 2: 1530 articles
Pillar 3: 278 articles
Pillar 4: 1511 articles
Pillar 5: 472 articles
Pillar 6: 453 articles
Pillar 7: 480 articles
Pillar 8: 1394 articles


## Saving gathered data

In [41]:
target_vars = [
    "id", "link", "domain_url", "published_date",
    "title_trans", "description_trans", "content_trans", "summary", "impact_score",
    "pillar_1", "pillar_2", "pillar_3", "pillar_4", "pillar_5", "pillar_6", "pillar_7", "pillar_8",
    "associated_pillar"
]
master = pd.concat(country_data).loc[:,target_vars]
master.to_parquet(f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-summarization/{country}/{country}_master.parquet.gzip", compression="gzip")

## Defining chain

In [42]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
}

In [43]:
class JSONOutputParser(BaseOutputParser):
    def parse(self, text: str):
        """
        Parse the output of an LLM call to a valid JSON format.
        """
        return json.loads(text.replace('```json', '').replace('```', ''), strict=False)

In [44]:
def summarize_pillar(summaries, pillar):
    """
    This function takes a list of summaries and it sends a call to Google's Gemini asking for a list of
    important events to take into account for a specific pillar of the Rule of Law.
    """
    
    idx = str(pillar)

    # Setting up the Prompt Template
    chat_prompt = ChatPromptTemplate.from_messages([
                    ("human", pts.pillar_summary_prompt)
                ])

    # Defining our chain
    chain_gemini = chat_prompt | ChatGoogleGenerativeAI(model = "gemini-pro",
                                                        temperature = 0.25, 
                                                        safety_settings = safety_settings,
                                                        convert_system_message_to_human = True) | JSONOutputParser()
    
    try: 
        llm_response = chain_gemini.invoke({
            "summaries"      : summaries,
            "pillar_name"    : pts.pillar_names[idx],
            "pillar_bullets" : pts.pillar_bullets[idx]
        })
        status = True
        time.sleep(1)   # We need to slow down the calls. given that the Gemini API has a limit of 60 calls per second

    # The API can still block some of our prompts due to undefined reasons. Sadly, we can't do anything about it, so we
    # predefine the outcome    
    except (BlockedPromptException, StopCandidateException):
        print("Prompt BLOCKED")
        status = False
    
    except JSONDecodeError:
        print("Decode error... trying again...")
        try: 
            llm_response = chain_gemini.invoke({
                "summaries"      : summaries,
                "pillar_name"    : pts.pillar_names[idx],
                "pillar_bullets" : pts.pillar_bullets[idx]
            })
            status = True
            time.sleep(1)

        except JSONDecodeError:
            print("Decode error... trying again...")
            try: 
                llm_response = chain_gemini.invoke({
                    "summaries"      : summaries,
                    "pillar_name"    : pts.pillar_names[idx],
                    "pillar_bullets" : pts.pillar_bullets[idx]
                })
                status = True
                time.sleep(1)

            except JSONDecodeError:
                print("Failed. Skipping...")
                status = False

    # We use the STATUS variable to throw an outcome to our call depending if our prompt was blocked or not
    if status == True:
        outcome = [llm_response["list_of_events"]]

    else:
        outcome = "Skipped list"

    return outcome

## Sending calls

In [45]:
def split_summaries(summaries):
    """
    This function takes a list of summaries and provides a text compiling all of them but taking into account the 
    token limit for using the Gemini Pro 1.0 API
    """

    idx = 0
    segments = [[]]
    total_count = 0

    for text in summaries:
        text_length = len(text.split())

        if total_count + text_length < 22500:
            segments[idx].append(text)
            total_count = total_count + text_length
        else:
            segments.append([])
            idx = idx + 1
            total_count = 0
            segments[idx].append(text)
            total_count = total_count + text_length

    outcome = ["- "+"\n- ".join(segment) for segment in segments]

    return outcome


In [46]:
summaries = {}
for p in range(1,9):

    article_list   = country_data[p-1]["summary"].to_list()
    limited_chunks = split_summaries(article_list)
    bullet_points  = [summarize_pillar(x, p) for x in limited_chunks]
    pillar_summ    = "/n".join([item for sublist in bullet_points for item in sublist])
    summaries.update({f"Pillar {p}" : pillar_summ})

Decode error... trying again...
Decode error... trying again...


## Saving summaries as JSON files

In [48]:
file_path = f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-summarization/Overview/{country}_overviewSummaries.json"
with open(file_path, "w") as json_file:
    json.dump(summaries, json_file)

In [47]:
summaries

{'Pillar 1': "- Hungary's investigation into the suspicious financing of the 99 Movement, a political organization founded by Budapest Mayor Gergely Karácsony.\n\n\n- The European Union's consideration of legal action against Hungary over concerns related to LGBTQ rights, academic freedom, and asylum rights.\n\n\n- The Hungarian government's reduction of the number of guest workers allowed in the country from 81,000 to 65,000 and the creation of a list of 300 occupations that are closed to foreign workers.\n\n\n- The European Parliament's debate on Hungary's frozen EU funds, with President Ursula von der Leyen stating that Hungary's justice system reforms comply with EU requests, leading to the release of some funds.\n\n\n- The Hungarian government's passage of a law that restricts the activities of NGOs, including those that receive funding from George Soros.\n\n\n- The European Parliament's vote on a motion expressing concerns about Hungary's compliance with EU rule of law principles